In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import subprocess, os, csv
from torch import nn
import torch, torchaudio, statistics
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import pandas as pd
import warnings, shutil
import datetime
import matplotlib.pyplot as plt

In [ ]:
# suppress all warnings
warnings.filterwarnings("ignore")
# suppress only the UserWarning with message "At least one mel filterbank has all zero values."
warnings.filterwarnings("ignore")

In [ ]:
# Model flow
class define_model(nn.Module):

    # Define layers
    def __init__(self, num_emotions):
        super().__init__()

        transformer_layer = nn.TransformerEncoderLayer(
            d_model=40, #####################
            nhead=4,
            dim_feedforward=512,
            dropout=0.4,
            activation='relu'
        )
        self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=4)

        #maxpool: reshape (width, height)
        #conv: reshape (channel)
        conv2d_layer = nn.Sequential(
            nn.Conv2d(
                in_channels=2,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(16),#######
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.3),
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(32),#######
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(64),#######
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
        )
        self.conv2Dblock1 = nn.Sequential(
            nn.Conv2d(
                in_channels=2,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.3),
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
        )
        self.conv2Dblock2 = conv2d_layer

        self.fc1_layer = nn.Linear(1792*2+40, 1800)
        self.act1 = nn.ReLU()
#         self.fc2_layer = nn.Linear(1800, 9)
#         self.act1 = nn.ReLU()
        self.fc2_layer = nn.Linear(1800, num_emotions)
        self.softmax_out = nn.Softmax(dim=1)

    def forward(self, x):
        conv2d_embedding1 = self.conv2Dblock1(x)
        conv2d_embedding1 = torch.flatten(conv2d_embedding1, start_dim = 1)

        conv2d_embedding2 = self.conv2Dblock2(x)
        conv2d_embedding2 = torch.flatten(conv2d_embedding2, start_dim = 1)

        x_maxpool = self.transformer_maxpool(x)
        # x_maxpool_reduced = torch.squeeze(x_maxpool,1) 
        x_maxpool_reduced = x_maxpool.resize(x_maxpool.shape[0], 40, 450) 
        x = x_maxpool_reduced.permute(2,0,1) #rearrange dims
        transformer_output = self.transformer_encoder(x)
        transformer_embedding = torch.mean(transformer_output, dim = 0)

        complete_embedding = torch.cat([conv2d_embedding1, conv2d_embedding2, transformer_embedding], dim = 1)
        fc1 = self.fc1_layer(complete_embedding)
        ac1 = self.act1(fc1)
#         fc2 = self.fc2_layer(ac1)
#         ac2 = self.act1(fc2)
        output_logits = self.fc2_layer(ac1)
        output_softmax = self.softmax_out(output_logits)
        return output_logits, output_softmax

In [ ]:
class preprocess_real_data():
    def __init__(self,file):
      self.file = file
    def mp4_to_wav(self, file):
        output_file = file.replace('input/real-data/Vietnamese/raw', 'working/wav_datas')
        output_file = output_file.split('.')[0] + ".wav"
        subprocess.call(['ffmpeg', '-i', file, output_file])
        return output_file
    def remove_noise(self, file):
        # Detect non-silent parts of the audio
        sound_file = AudioSegment.from_wav(file)
        non_sil_times = detect_nonsilent(sound_file, min_silence_len=400, silence_thresh=sound_file.dBFS * 0.65)
        # Concatenate the non-silent parts of the audio
        if len(non_sil_times) > 0:
            non_sil_times_concat = [non_sil_times[0]]
            if len(non_sil_times) > 1:
                for t in non_sil_times[1:]:
                    if t[0] - non_sil_times_concat[-1][1] < 100:
                        non_sil_times_concat[-1] = (non_sil_times_concat[-1][0], t[1])
                    else:
                        non_sil_times_concat.append(t)
            new_audio = sound_file[non_sil_times_concat[0][0]:non_sil_times_concat[0][1]]
            for t in non_sil_times_concat[1:]:
                new_audio += sound_file[t[0]:t[1]]
        else:
            new_audio = sound_file

        # Export the new audio file
        file_name = file.replace('wav_datas', 'denoised_datas')
        file_name = file_name[:-4]+'_denoised.wav'
        new_audio.export(file_name, format="wav")
        return file_name
    def resize(self, file):
        tensor = torch.load(file)
        n = tensor.shape[-1]//900 + 1
        sample = torch.zeros((2,40,n * 900))
        sample[:,:,:tensor.shape[-1]] = tensor
        sample = sample.reshape((2,40,900,n)).permute(3,0,1,2)
        torch.save(sample,file)
#         sample = torch.transpose(torch.transpose(sample1.reshape((40,500,n)),dim0=0, dim1=2),dim0=1, dim1=2).reshape((n,1,40,500))
        return sample
    def save_tensor_file(self, file):
        waveform, sample_rate = torchaudio.load(file)
        transform = torchaudio.transforms.MFCC(sample_rate=sample_rate)
        mfcc = transform(waveform)
        file_name = file[:-4]+'.pt'
        torch.save(mfcc,file_name)
        return file_name
    def complete_preprocessing(self, file):
      file = self.mp4_to_wav(file)
      file = self.remove_noise(file)
      file = self.save_tensor_file(file)
      tensor = self.resize(file)
      return file, tensor

In [ ]:
def load_model(model_path):
    model = define_model(3)
    model.eval()
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
#     model.load_state_dict(torch.load(model_path))
    return model

In [ ]:
# model = load_model('/kaggle/input/dpl-project/model93.pt')
# # model = torch.load('/kaggle/input/dpl-project/model_update1.pt', map_location='cpu')
# # model.load_state_dict(torch.load('model.pt'))
# # model.parameters()
# num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(f'Total number of trainable parameters: {num_params}')

In [ ]:
import wave

def get_wav_duration(file_path):
    with wave.open(file_path, 'rb') as wav_file:
        # Get the frame rate (samples per second) and total number of frames
        frame_rate = wav_file.getframerate()
        total_frames = wav_file.getnframes()

        # Calculate the duration in seconds
        duration = total_frames / float(frame_rate)
    
    return duration

In [ ]:
def predict_real_time(file_sample, model, preprocesser):
    _, tensor = preprocesser.complete_preprocessing(file_sample)
    output_logit, output_softmax = model(tensor)
    output_softmax = torch.argmax(output_softmax, dim=1)
    emotion_dict = {0: 'positive',
                    1: 'neutral',
                    2: 'negative'}
    labels = [emotion_dict[int(value)] for value in output_softmax]
    
    final_output = max(set(output_softmax.tolist()), key=output_softmax.tolist().count)
    label = emotion_dict[final_output]
    
    duration_seconds = get_wav_duration(file_sample)
    range_ = duration_seconds/ len(labels)
    a = datetime.time(0, 0)
    list_labels = list()
    delta = datetime.timedelta(seconds=round(range_, 5))
    for label in labels:
        new_time = datetime.datetime.combine(datetime.datetime.today(), a) + delta
        list_labels.append(str(a)[3:8]+'-'+str(new_time.time())[3:8]+'_'+str(label))
        a = new_time.time()
    return(list_labels)

In [ ]:
def waveform(path):
    waveform, sr = torchaudio.load(path)
    plt.figure(figsize = (10,4))
    plt.plot(waveform.numpy().T)
    plt.title('Waveform')
    plt.xlabel('Time')
    plt.ylabel('Frequency')

In [ ]:
# preprocesser = preprocess_real_data(0)
# model = load_model('/kaggle/input/models/final.pt')
# file_sample = '/kaggle/input/dpl-project/test_data/raw_datas/negative/Nhat_01.wav'
# waveform(file_sample)
# predict_real_time(file_sample, model, preprocesser)

In [ ]:
def predict_1_sample(file_sample, model):
    tensor = torch.load(file_sample)
    output_logit, output_softmax = model(tensor)
    output_softmax = torch.argmax(output_softmax, dim=1)
    final_output = max(set(output_softmax.tolist()), key=output_softmax.tolist().count)
    emotion_dict = {0: 'positive',
                    1: 'neutrual',
                    2: 'negative'}
    label = emotion_dict[final_output]
    return final_output, label

In [ ]:
def extract_label_file(root):
  header = ['sessionID','labels','path']
  emotion_dict = {
      'positive': 0,
      'neutrual': 1,
      'negative': 2
  }
  with open('test_real_data.csv',mode='w',newline='') as file:
          write = csv.writer(file)
          write.writerow(header)
  df = pd.read_csv('test_real_data.csv')
  for file in os.listdir(root):
          for sample in os.listdir(os.path.join(root, file)):
              processer = preprocess_real_data(0)
              path_sample, tensor = processer.complete_preprocessing(os.path.join(root, file, sample))
              sample_info = {'sessionID': sample, 'labels': emotion_dict[file],'path': path_sample}
              df = pd.concat([df, pd.DataFrame([sample_info])], ignore_index=True)
  df.to_csv('test_real_data.csv')

In [ ]:
def test_real_data(model_file):
  df = pd.read_csv('/kaggle/working/test_real_data.csv')
  df = df.sort_values('labels')
  model = load_model(model_file)
  preprocesser = preprocess_real_data('ok')
  y = df.labels
  y = torch.as_tensor(y)
  y_hat = torch.zeros(len(y))
  ind_sample = 0
  for sample in df.path:
    final_output, label = predict_1_sample(sample, model)
    y_hat[ind_sample] = final_output
    ind_sample += 1
  df.to_csv('/kaggle/working/test_real_data.csv')
  return y, y_hat

In [ ]:
# shutil.rmtree('/kaggle/working/wav_datas')
# shutil.rmtree('/kaggle/working/denoised_datas')
# folder_root = ['denoised_datas','wav_datas']
# folder = ['positive','neutrual','negative']
# for fold in folder_root:
#     os.makedirs(os.path.join('/kaggle/working/',fold))
#     for fold_ in folder:
#         os.makedirs(os.path.join('/kaggle/working/',fold,fold_))

In [ ]:
root = '/kaggle/input/real-data/Vietnamese/raw'
extract_label_file(root)
model_file = '/kaggle/input/models/final.pt'
y, y_hat = test_real_data(model_file)

In [ ]:
def metric(y, y_hat):
  df = pd.read_csv('/kaggle/working/test_real_data.csv')
  labels = {'positive':0,'neutral':0,'negative':0}
  start_num = 0
  for i, label in enumerate(labels):
    end_num = start_num+len(df.loc[df.labels == i])
    TP = torch.sum(y[start_num:end_num] == y_hat[start_num:end_num])
    the_last = torch.cat((y_hat[:start_num],y_hat[end_num:]))
    FP = torch.count_nonzero( the_last == i)
    FN = torch.count_nonzero( y_hat[start_num:end_num] != i)
#     print('Precision of',label, ': ', TP/(TP+FP))
    F1 = 2*TP / (2*TP+FP+FN)
    labels[label] = round(F1.item(),2)
    start_num = end_num
  accuracy = torch.sum(y==y_hat)/float(len(y))
  print('F1_score of every label: ', labels)
  print('Average F1: ',round(statistics.mean(labels.values()),2))
  print('Accuracy: ', accuracy)

In [ ]:
metric(y, y_hat)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y, y_hat)
print(cm)

In [ ]:
cmn = (cm.astype('float') / cm.sum(axis=1)[:, np.newaxis])*100

ax = plt.subplots(figsize=(8, 5.5))[1]
sns.heatmap(cmn, cmap='flare', annot=True, square=True, linecolor='black', linewidths=0.75, ax = ax, fmt = '.2f', annot_kws={'size': 16})
ax.set_xlabel('Predicted', fontsize=18, fontweight='bold')
ax.xaxis.set_label_position('bottom')
ax.xaxis.set_ticklabels(["positive", "neutral", "negative"], fontsize=16)
ax.set_ylabel('Ground Truth', fontsize=18, fontweight='bold')
ax.yaxis.set_ticklabels(["positive", "neutral", "negative"], fontsize=16)
plt.tight_layout()
plt.show()

![](http://)